In [ ]:
#%pip install gensim

In [ ]:
#%pip install spacy

In [ ]:
#%pip install pyLDAvis 

In [46]:
#Imports
import pandas as pd
import pickle
import os
import nltk
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('words')
import pandas as pd
from nltk.tokenize import PunktSentenceTokenizer, RegexpTokenizer
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import CountVectorizer
from nltk.stem import PorterStemmer
import matplotlib.pyplot as plt

import gensim
from gensim.models.ldamodel import LdaModel
import gensim.corpora as corpora
from gensim.models import CoherenceModel
from gensim import matutils

import spacy

import pyLDAvis
import pyLDAvis.gensim_models

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\louis\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\louis\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package words to
[nltk_data]     C:\Users\louis\AppData\Roaming\nltk_data...
[nltk_data]   Package words is already up-to-date!


In [2]:
#Import soptify + genius dataset with song lyrics
df_lyrics = pd.read_pickle("df_rap_7k.pickle")

In [3]:
df_lyrics.head()

,track_id,album_id,artist_id,duration_ms,flg_explicit,url,track_name,track_popularity,track_type,track_danceability,...,track_release_year,track_release_decade,artist_name,artist_genres,artist_followers,artist_popularity,lyrics,lyrics_clean,lyrics_clean_split,lyrics_wordlen
0,3SUiKUnZlKYliid0ZlCmUY,0uYanB3WWNU2lT9AYxCstH,3M1quhETLChtt9NHKVaajX,189405,True,https://api.spotify.com/v1/tracks/3SUiKUnZlKYl...,Detroit Flow,44,track,0.793,...,2020,20,Quin Nfn,"[austin hip hop, southern hip hop, vapor trap]",257151,61,"Detroit Flow Lyrics\n(Ayy, DJ Showoutt, fuck g...",detroit flow fuck hold really like sound beats...,"[detroit, flow, fuck, hold, really, like, soun...",244
1,2jdWfnwHSZLuBFzsA6axWt,0iNKw3iqIEy2dh7RUdKzjY,0nQlnRkAhiWnSjTmB4JkfK,273099,True,https://api.spotify.com/v1/tracks/2jdWfnwHSZLu...,Handz in da Air,8,track,0.826,...,2018,10,Sons of Light,[],141,7,Pet Sound LyricsTurbid:\nYes it's Turbid and I...,pet sound yes turbid like reclaim freely like ...,"[pet, sound, yes, turbid, like, reclaim, freel...",209
2,2vcLUuRT3sqXnPJaIX5w5c,0VptfDjz26NkJS4b7bjmNK,0FtsMKmZEq8fBWqdSOWtqp,236619,True,https://api.spotify.com/v1/tracks/2vcLUuRT3sqX...,Substance (We Woke Up),20,track,0.731,...,2021,20,03 Greedo,"[cali rap, melodic rap, trap, vapor trap]",317202,71,"Substance (We Woke Up) Lyrics\nYeah, 03\nYeah,...",substance woke yeah yeah yeah yeah beat boy ta...,"[substance, woke, yeah, yeah, yeah, yeah, beat...",269
3,1r0faljjM2b876iNoaDUh5,0XAIjjN5qxViVS0Y5fYkar,0c173mlxpT3dSFRgMO8XPh,287320,True,https://api.spotify.com/v1/tracks/1r0faljjM2b8...,Sacrifices,56,track,0.779,...,2017,10,Big Sean,"[detroit hip hop, hip hop, pop, pop rap, rap, ...",9276030,84,"Sacrifices Lyrics\nBrrrt!\nI look up, so many ...",look many shit many young metro trust gon shoo...,"[look, many, shit, many, young, metro, trust, ...",421
4,3yhatBRYsMwuozCdQsGiTS,2t70lfxoIbKZHKFKN9ZnqE,0auu2itHTxEdAMRHvx7CyG,283200,False,https://api.spotify.com/v1/tracks/3yhatBRYsMwu...,Hard Core Rap Act,28,track,0.813,...,1995,90,Das EFX,"[east coast hip hop, gangster rap, hardcore hi...",301411,60,Hardcore Rap Act LyricsIntro/Hook (x2)\n(*The ...,hardcore rap act hook hardcore rap act back ha...,"[hardcore, rap, act, hook, hardcore, rap, act,...",267


In [4]:
#Create lemmatizer, stemmer and define stopwords

word_tokenizer=RegexpTokenizer(r'\w+')

wordnet_lemmatizer = WordNetLemmatizer()

stemmer = PorterStemmer()

stopwords_nltk = set(stopwords.words('english'))



In [5]:
#Save lyrics
temp = df_lyrics["lyrics_clean"].copy()

In [6]:
### Function to lemmatize tokenized documents (song lyrics here)
def lemmatizer(df):
    
    lemmed = []
    
    for song in df.lyrics_clean_split:
        lemmed.append([wordnet_lemmatizer.lemmatize(word) for word in song])
    return lemmed

In [11]:
### Fucntion to get dictionary and bag of words
#Returns dictionary and bow corpus
def get_dict_and_bow(data):
    
    #Create dictionary of words: mapping between word and id
    id2word = corpora.Dictionary(data)

    texts = data
    
    #Transform dictionary into bag of words
    corpus = [id2word.doc2bow(text) for text in texts]
    
    return id2word, corpus

In [19]:
### Function to get coherence score for particular model, dict and bow
#Returns the coherence model
def get_coherence(model, text, id_dict, method='c_v'):
    coherence_model = CoherenceModel(model=model, texts=text, dictionary=id_dict, coherence=method)
    coherence_lda = coherence_model.get_coherence()
    print(coherence_lda)
    return coherence_model

In [9]:
lyrics_lemmatized_all = lemmatizer(df_lyrics)

In [13]:
id2word_all, corpus_all = get_dict_and_bow(lyrics_lemmatized_all)

In [82]:
#Fit LDA model
lda_model_all = LdaModel(corpus = corpus_all,
                    id2word = id2word_all,
                    num_topics = 10, 
                    random_state = 100,
                    passes=2,
                    alpha=0.5,
                    per_word_topics=True)

In [83]:
#Show the LDA topics: topic 0
lda_model_all.show_topic(topicid=0, topn=20)


[('feat', 0.18116966),
 ('young', 0.021290036),
 ('thug', 0.015670693),
 ('baby', 0.0130217355),
 ('scott', 0.011798486),
 ('travis', 0.011284275),
 ('big', 0.009115704),
 ('wayne', 0.008911655),
 ('future', 0.008787202),
 ('drake', 0.007595347),
 ('ross', 0.00697768),
 ('rick', 0.006930346),
 ('west', 0.006473689),
 ('life', 0.006182214),
 ('cole', 0.0061672707),
 ('kid', 0.006068576),
 ('mill', 0.0053539877),
 ('savage', 0.0052777883),
 ('love', 0.005254062),
 ('polo', 0.005148006)]

In [63]:
get_document_topics = [lda_model_all.get_document_topics(item,minimum_probability =0) for item in corpus_all]


In [74]:
get_document_topics[0][0][1]

0.0024679585

In [75]:
df_list = []
for i in range(len(get_document_topics)):
    temp_list = []
    for j in range(len(get_document_topics[i])):
        temp_list.append(get_document_topics[i][j][1])
    df_list.append(temp_list)

In [77]:
df_list = pd.DataFrame(get_document_topics)

In [78]:
df_list

,0,1,2,3,4,5
0,"(0, 0.0024679585)","(1, 0.10925192)","(2, 0.0023709817)","(3, 0.015697883)","(4, 0.22399178)","(5, 0.6462195)"
1,"(0, 0.025928663)","(1, 0.4994075)","(2, 0.061931994)","(3, 0.05382823)","(4, 0.32867005)","(5, 0.030233575)"
2,"(0, 0.039953813)","(1, 0.40987605)","(2, 0.06590306)","(3, 0.016491208)","(4, 0.046070013)","(5, 0.42170587)"
3,"(0, 0.016093653)","(1, 0.52810246)","(2, 0.009410793)","(3, 0.013160394)","(4, 0.0019852947)","(5, 0.43124738)"
4,"(0, 0.0025316712)","(1, 0.10623469)","(2, 0.002282761)","(3, 0.002587677)","(4, 0.77196646)","(5, 0.11439677)"
...,...,...,...,...,...,...
6778,"(0, 0.011117341)","(1, 0.60917556)","(2, 0.003573124)","(3, 0.06046473)","(4, 0.31030676)","(5, 0.00536245)"
6779,"(0, 0.0031546617)","(1, 0.3339195)","(2, 0.24324138)","(3, 0.0031154535)","(4, 0.23086584)","(5, 0.18570313)"
6780,"(0, 0.025961924)","(1, 0.53292924)","(2, 0.017295586)","(3, 0.029956512)","(4, 0.37765217)","(5, 0.01620454)"
6781,"(0, 0.9818451)","(1, 0.0021898889)","(2, 0.0024528003)","(3, 0.0027097869)","(4, 0.0021075448)","(5, 0.008694945)"


In [57]:
print(get_document_topics[0])

[(1, 0.10924671), (3, 0.015698135), (4, 0.2239934), (5, 0.64622283)]


In [84]:
#Get coherence score
coherence_model_all = get_coherence(lda_model_all, text = lyrics_lemmatized_all, id_dict=id2word_all)

0.3805367693039513


In [85]:
# Visualize the topics
pyLDAvis.enable_notebook()
vis_all = pyLDAvis.gensim_models.prepare(lda_model_all, corpus_all, id2word_all)
vis_all

C:\Users\louis\anaconda3\envs\INSY669\lib\site-packages\pyLDAvis\_prepare.py:246: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  default_term_info = default_term_info.sort_values(


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
1     -0.068399  0.043209       1        1  27.814041
5     -0.110959 -0.018589       2        1  20.631473
4     -0.105995  0.053495       3        1  12.453932
3      0.191677 -0.010147       4        1   9.392729
0      0.268595 -0.002937       5        1   8.694022
6     -0.053250  0.052964       6        1   8.545809
8     -0.061362  0.107922       7        1   6.679543
2      0.030997  0.007575       8        1   2.410628
7     -0.000804 -0.029279       9        1   1.764344
9     -0.090500 -0.204213      10        1   1.613478, topic_info=      Term          Freq         Total Category  logprob  loglift
719   feat  35452.000000  35452.000000  Default  30.0000  30.0000
282   yeah  29157.000000  29157.000000  Default  29.0000  29.0000
321    ooh   4371.000000   4371.000000  Default  28.0000  28.0000
6    bitch  13306.000000  13306.000000  Default  27.0000  27.0000
81    love  10419.000000  10419.000000  Default  26.0000  26.0000
..     ...           ...           ...      ...      ...      ...
410   made    146.358036   3341.921864  Topic10  -5.2585   0.9985
121    run    132.778391   3352.627997  Topic10  -5.3559   0.8980
137   take    145.752071   6327.035863  Topic10  -5.2627   0.3561
76    like    139.004781  34375.355080  Topic10  -5.3101  -1.3838
75     let    115.924542   8418.782544  Topic10  -5.4917  -0.1585

[862 rows x 6 columns], token_table=       Topic      Freq      Term
term                            
7101       1  0.006809  acoustic
7101       4  0.892004  acoustic
7101       5  0.061283  acoustic
7101       6  0.020428  acoustic
7101       8  0.013618  acoustic
...      ...       ...       ...
16278      9  0.855716  zanzibar
9532       1  0.038923       zel
9532       5  0.077846       zel
9532       6  0.038923       zel
9532      10  0.817380       zel

[3682 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[2, 6, 5, 4, 1, 7, 9, 3, 8, 10])

# Topics by 5-year segments

In [ ]:
#We will hardcode the year segments - call them periods
df_lyrics.track_release_year.value_counts().sort_index().plot(kind='barh',figsize=(10, 10))

Bar plot shows that we can start the first period in 1990. Periods will be of x years.

In [34]:
#Create dictionary with periods as lists.
def create_periods(x=5):
    period_dict = {}
    max_year = df_lyrics.track_release_year.max()
    min_year = 1990
    
    i = 0
    while i < (max_year - 1990)/5:
        i +=1
        period_dict[i] = []
        period_dict[i].append(min_year)
        min_year += x - 1
        period_dict[i].append(min_year)
        min_year += 1
    return period_dict

In [35]:
period_dict = create_periods(5)
period_dict

{1: [1990, 1994],
 2: [1995, 1999],
 3: [2000, 2004],
 4: [2005, 2009],
 5: [2010, 2014],
 6: [2015, 2019],
 7: [2020, 2024]}

In [44]:
#Loop through different periods, create dictionary, bow, model, coherence and visualization for all

for i in range(len(period_dict)):
    
    #Temporary df with songs from that period
    df_temp = df_lyrics

0
1
2
3
4
5
6


# Topics for Artist: Nas

In [23]:
df_nas = df_lyrics[df_lyrics.artist_name == 'Nas']

In [24]:
#Lemmtaize lyrics
lyrics_lemmatized_nas = lemmatizer(df_nas)

In [25]:
#Get id-word dict, bow
id2word_nas, corpus_nas = get_dict_and_bow(lyrics_lemmatized_nas)

In [37]:
#Fit LDA model for Nas
lda_model_nas = LdaModel(corpus=corpus_nas,
                    id2word=id2word_nas,
                    num_topics=4, 
                    random_state=100,
                    chunksize=200,
                    passes=2,
                    alpha=0.5,
                    per_word_topics=True)

In [38]:
#Get coherence score for Nas
coherence_model_nas = get_coherence(lda_model_nas, text = lyrics_lemmatized_nas, id_dict = id2word_nas)

0.30729217061263225


In [39]:
#Visualize the topics
pyLDAvis.enable_notebook()
vis_nas = pyLDAvis.gensim_models.prepare(lda_model_nas, corpus_nas, id2word_nas)
vis_nas

C:\Users\louis\anaconda3\envs\INSY669\lib\site-packages\pyLDAvis\_prepare.py:246: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  default_term_info = default_term_info.sort_values(


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
1      0.027300  0.022792       1        1  32.164184
2     -0.100008 -0.004989       2        1  28.027861
0      0.046941 -0.031284       3        1  24.734897
3      0.025767  0.013481       4        1  15.073058, topic_info=       Term         Freq        Total Category  logprob  loglift
935    feat  2826.000000  2826.000000  Default  30.0000  30.0000
216   world   331.000000   331.000000  Default  29.0000  29.0000
150    like   642.000000   642.000000  Default  28.0000  28.0000
40      man   324.000000   324.000000  Default  27.0000  27.0000
60     shit   317.000000   317.000000  Default  26.0000  26.0000
..      ...          ...          ...      ...      ...      ...
33     know    38.519328   370.501922   Topic4  -5.3943  -0.3714
59      see    32.508433   233.566800   Topic4  -5.5640  -0.0797
570  street    33.111804   259.317210   Topic4  -5.5456  -0.1659
149     let    29.764809   197.840398   Topic4  -5.6522  -0.0019
253    fuck    27.862694   178.785826   Topic4  -5.7182   0.0334

[349 rows x 6 columns], token_table=      Topic      Freq    Term
term                         
1467      1  0.110396  action
1467      2  0.728614  action
1467      3  0.066238  action
1467      4  0.088317  action
2031      1  0.245510   aesop
...     ...       ...     ...
77        4  0.134005    yeah
2030      1  0.107735  zombie
2030      2  0.754144  zombie
2030      3  0.107735  zombie
2030      4  0.053867  zombie

[885 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[2, 3, 1, 4])

## Topics by Period

In [ ]:
dict_epoch_nas = get_epochs(df_nas, 5)

In [ ]:
# Get topics by epoch

topic_nas_list = []
artists_irrelevant = []

for i in range(len(dict_epoch_nas)):
    
    #Only get songs for that epoch
    df_temp = df_nas[(df_nas.track_release_year >= dict_epoch_nas[i][0])
                        & (df_nas.track_release_year <= dict_epoch_nas[i][1])]
    
    #Check to see how many songs are in df
    print(f"There are {len(df_temp)} songs in epoch {i}: {dict_epoch_nas[i][0]} to {dict_epoch_nas[i][1]}")
    
    #Break loop if df is empty
    if df_temp.empty:
        print('Break: no songs')
        break
        
    #Vectorize lyrics
    vec_lyrics, count_vectorizer = count_vectorize_lyrics(df_temp, tokenize_text_lemma)
    
    #Get topics and artists df
    temp_topics, temp_artists = get_topics(df_temp, vec_lyrics, vectorizer = count_vectorizer, check_convergence=True)
    
    #Sort topic coeffs
    temp_topics_sorted = get_sorted_coeffs(temp_topics)
    
    topic_nas_list.append(temp_topics_sorted)
    artists_irrelevant.append(temp_artists)

In [ ]:
dict_epoch_nas

In [ ]:
topic_nas_list[0][0].head(40).plot.bar()

In [ ]:
topic_nas_list[0][1].head(40).plot.bar()

In [ ]:
topic_nas_list[0][2].head(40).plot.bar()

In [ ]:
topic_nas_list[0][3].head(40).plot.bar()

In [ ]:
topic_nas_list[0][4].head(40).plot.bar()